In [1]:
from tkinter import *
from tkinter import scrolledtext

In [80]:
from os import listdir, path, getcwd, remove
from collections import defaultdict

In [3]:
#from datetime import datetime
#import time

In [95]:
# запуск

MyApp().window.mainloop()

In [93]:
class MyApp:
    
    def __init__(self):
        abs_path = path.join(getcwd(), "books")
        self.book_list = listdir(abs_path)
        self.file_content_list, self.file_content_list_lower = self.file_content_list_func(self.book_list)
        # file_content_list, file_content_list_lower - это словари {книга:контент}
        
        self.window = Tk()
        self.window.title("Поиск книг")
        
        self.label = Label(self.window, 
            text="Введите ключевые слова (можно без окончаний) или словосочетание через запятую:", 
                           width = 100, height = 2, font = 15)
        self.label_request_title = Label(self.window, text="", fg="blue", 
                           width = 80, height = 2, font = 15)
        self.label_request = Label(self.window, text="", 
                           width = 80, height = 2, font = 15)
        self.label_result_title = Label(self.window, text="", fg="blue",
                           width = 80, height = 2, font = 15)        
        self.text_area = scrolledtext.ScrolledText(self.window, 
                           width = 80, height = 30, font = 10)
        self.entry = Entry(self.window,
                           width = 80, font = 15)
        self.button = Button(
            self.window, text="Поиск", command = self.print_request)
        
        
        self.label.grid(
            row=0, column = 0, columnspan = 2)
        self.entry.grid(
            row=1, column = 0)
        self.button.grid(
            row=1, column = 1)        
        self.label_request_title.grid(
            row=2, column = 0)
        self.label_request.grid(
            row=3, column = 0)        
        self.label_result_title.grid(
            row=4, column = 0)        
        self.text_area.grid(
            row=5, column = 0, pady = 20, padx = 20)        
        
        
    def file_content_list_func(self, book_list):
        self.content_list = {}
        self.content_list_lower = {}
        for file_name in self.book_list:
            self.content = self._file_read(file_name)[0]
            self.content_list[file_name] = self.content
            self.content_lower = self._file_read(file_name)[1]
            self.content_list_lower[file_name] = self.content_lower         
        return self.content_list, self.content_list_lower           # это словари
    
       
    def _file_read(self, file_name):
        abs_path = path.join(getcwd(), f"books\\{file_name}")
        try:
            with open(abs_path, "r") as file:
                self.content = file.read().replace("\n", "")
        except UnicodeDecodeError:
            with open(abs_path, "r", encoding="UTF-8") as file:
                self.content = file.read().replace("\n", "")
        self.content_lower = self.content.lower()
        return self.content, self.content_lower                     # это строки
    
    
    def print_request(self):
        
        key_word_string = self.entry.get()
        
        self.entry.delete(0, 'end')
        self.text_area.delete(0.0, 'end')
        
        self.label_request_title['text'] = f"Поиск по запросу:"
        self.label_request['text'] = f"{key_word_string}"
        
        key_word_list = key_word_string.lower().split(", ")
        
        # обработка вывода:    
        result_string = Librarian6(key_word_list, self.file_content_list_lower).file_search()        # Librarian6
        
        text_context = ""
        self.dict_out = defaultdict(str) 
        for book_title, index_list in result_string[0].items():
            for i in index_list:                
                if i > 50:
                    self.dict_out[book_title] += f"{self.file_content_list[book_title][(i - 50):(i + 50)]} -+++- "
                else:
                    self.dict_out[book_title] += f"{self.file_content_list[book_title][:(i + 100)]} -+++- "
            text_context += f"{book_title}\n{self.dict_out[book_title]}\n\n"
        
        # вывод результа:
        self.label_result_title['text'] = f"Результат поиска: {result_string[1]}"        
        self.text_area.insert(INSERT, f"{text_context}")

In [94]:
class Librarian6:
    
    def __init__(self, key_word_list, file_content_list):
        self.key_word_list = key_word_list
        self.file_content_list = file_content_list
    
    
    def file_search(self):
        self.book_count = 0
        self.book_list_find = {}
        for book_title, content in self.file_content_list.items():
            self.key_word_list_find = []
            index_list = []
            flag = True
            for word in self.key_word_list:
                if flag == True:
                    i = content.find(word)
                    if i != -1:
                        self.key_word_list_find.append(word)
                        index_list.append(i)
                    else:
                        flag = False
            if self.key_word_list == self.key_word_list_find:
                self.book_list_find[book_title] = index_list
         
        self.book_count = len(self.book_list_find)
        
        return self.book_list_find, self.book_count